# Upload training-20210903T060329Z-001.zip file of model checkpoint in the '/content/' directory

In [ ]:
from google.colab import files

files.upload()

Or, Mount your Google drive to the VM

In [ ]:

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!cp '''Here goes path to training-20210903T060329Z-001.zip''' /content/

In [ ]:
drive.flush_and_unmount()

# Unzip the checkpoint directory

In [ ]:
# !unzip training-20210903T060329Z-001.zip
!unzip training.zip

# Download the dataset for testing

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [ ]:
!pip install -q kaggle

from google.colab import files

files.upload()

!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d grassknoted/asl-alphabet

!unzip asl-alphabet.zip

for c in 'ABCDEFGHIJKLMNOPQRSTUVWXYZ':
  os.mkdir('/content/asl_alphabet_test/asl_alphabet_test/'+c+'/')

os.mkdir('/content/asl_alphabet_test/asl_alphabet_test/'+'del'+'/')
os.mkdir('/content/asl_alphabet_test/asl_alphabet_test/'+'nothing'+'/')
os.mkdir('/content/asl_alphabet_test/asl_alphabet_test/'+'space'+'/')

for c in 'ABCDEFGHIJKLMNOPQRSTUVWXYZ':
  os.replace('/content/asl_alphabet_test/asl_alphabet_test/'+c+'_test.jpg', '/content/asl_alphabet_test/asl_alphabet_test/'+c+'/'+c+'.jpg')

os.replace('/content/asl_alphabet_test/asl_alphabet_test/nothing_test.jpg', '/content/asl_alphabet_test/asl_alphabet_test/nothing/nothing.jpg')
os.replace('/content/asl_alphabet_test/asl_alphabet_test/space_test.jpg', '/content/asl_alphabet_test/asl_alphabet_test/space/space.jpg')


# Load the data set off the disk

In [ ]:
import pathlib
train_data_dir = pathlib.Path('/content/asl_alphabet_train/asl_alphabet_train/')
test_data_dir = pathlib.Path('/content/asl_alphabet_test/asl_alphabet_test')

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  train_data_dir,
  image_size=(200, 200),
  batch_size=32)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
  test_data_dir,
  image_size=(200, 200),
  batch_size=32)

# Define model configuration

In [ ]:
def create_model():
  num_classes = 29

  model = Sequential([
    layers.experimental.preprocessing.Rescaling(1./255, input_shape=(200, 200, 3)),
    layers.Conv2D(16, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes)
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
  
  return model

# Load the model weights

In [ ]:
checkpoint_path = "/content/training/cp.ckpt"

model = create_model()
model.load_weights(checkpoint_path)

# Test the model


In [ ]:
test_loss, test_acc = model.evaluate(test_ds)

In [ ]:
test_loss

In [ ]:
test_acc

In [ ]:
probability_model = tf.keras.Sequential([model, 
                                         tf.keras.layers.Softmax()])

In [ ]:
predictions = probability_model.predict(test_ds)

In [ ]:
predictions

In [ ]:
class_names = test_ds.class_names
print(class_names)

In [ ]:
print(np.argmax(predictions))

In [ ]:
print(np.argmax(predictions[0]))

In [ ]:
print(np.argmax(predictions[1]))

In [ ]:
print(np.argmax(predictions[2]))

# Visualize the predictions

In [ ]:
def plot_image(i, predictions_array, true_label, img):
  true_label, img = true_label[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(img, cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'

  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                true_label),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
  true_label = true_label[i]
  plt.grid(False)
  plt.xticks(range(29))
  plt.yticks([])
  thisplot = plt.bar(range(29), predictions_array, color="#777777")
  plt.ylim([0, 1])
  predicted_label = np.argmax(predictions_array)

  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')


In [ ]:
images = list(test_data_dir.glob('*/*.jpg'))

In [ ]:
import re

labels = list()

for img in images:
  labels.append(re.findall(r'/(\w+)\.jpg', str(img))[0])

In [ ]:
labels = np.array([class_names.index(i) for i in labels])

In [ ]:
img = [PIL.Image.open(i) for i in images]

In [ ]:
image_list = np.array([np.array(i) for i in img])
image_list.shape

In [ ]:
test_predictions = probability_model.predict(image_list)

In [ ]:
num_rows = 14
num_cols = 2
num_images = num_rows*num_cols
plt.figure(figsize=(2*7*num_cols, 2*num_rows))
for j in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*j+1)
  plot_image(j, test_predictions[j], labels, image_list)
  plt.subplot(num_rows, 2*num_cols, 2*j+2)
  plot_value_array(j, test_predictions[j], labels)
plt.tight_layout()
plt.show()

In [ ]:
import cv2

In [ ]:
!pwd
image = cv2.imread('/content/A_test.jpg')


In [ ]:
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.figure(figsize=(20, 20))
plt.subplot(2, 2, 1)
plt.title("Original")
plt.imshow(image)
# image_scaled = cv2.resize(image, None, fx=0.75, fy=0.75)
# plt.subplot(2, 2, 2)
# plt.title("Scaling - Linear Interpolation")
# plt.imshow(image_scaled)
# img_scaled = cv2.resize(image, None, fx=2, fy=2, interpolation = cv2.INTER_CUBIC)
# plt.subplot(2, 2, 3)
# plt.title("Scaling - Cubic Interpolation")
# plt.imshow(img_scaled)
img_scaled = cv2.resize(image, (200, 200), interpolation = cv2.INTER_AREA)
plt.subplot(2, 2, 2)
plt.title("Scaling - Skewed Size")
plt.imshow(img_scaled)

Testing

In [ ]:
resizing_model = tf.keras.Sequential([
  layers.experimental.preprocessing.Resizing(200, 200, crop_to_aspect_ratio=False)
])

In [ ]:
sample_path = '/content/sample_images/*'
sample_ds = tf.data.Dataset.list_files(sample_path)
sample_file = next(iter(sample_ds))
image = tf.io.read_file(sample_file)
image = tf.image.decode_png(image)
image = tf.image.convert_image_dtype(image, tf.float32)

In [ ]:
plt.imshow(image)

In [ ]:
resized_image = resizing_model(image.numpy())
plt.imshow(resized_image)